In [ ]:
import pandas as pd
import torch.nn as nn
import os
from sklearn.model_selection import train_test_split
from PIL import Image
from torchvision import transforms
import torch
import random
import os

cwd = os.getcwd()
print("Current working directory:", cwd)

Current working directory: /sise/home/amitfoye/code_files/DL_A2


# Load The Dataset

### Paths

In [2]:
dataset_path = os.path.join(cwd, "lfw")
images_path = os.path.join(dataset_path, "lfw-deepfunneled", "lfw-deepfunneled")

match_dev_test_path = os.path.join(dataset_path, "matchpairsDevTest.csv")
match_dev_train_path = os.path.join(dataset_path, "matchpairsDevTrain.csv")
match_dev_val_path = os.path.join(dataset_path, "matchpairsDevVal.csv")

mismatch_dev_test_path = os.path.join(dataset_path, "mismatchpairsDevTest.csv")
mismatch_dev_train_path = os.path.join(dataset_path, "mismatchpairsDevTrain.csv")
mismatch_dev_val_path = os.path.join(dataset_path, "mismatchpairsDevVal.csv")


## Load The Data

In [3]:
match_dev_test_df = pd.read_csv(match_dev_test_path)
match_dev_train_df = pd.read_csv(match_dev_train_path)
mismatch_dev_test_df = pd.read_csv(mismatch_dev_test_path)
mismatch_dev_train_df = pd.read_csv(mismatch_dev_train_path)

print(mismatch_dev_test_df.head())

                    name1  imagenum1                         name2  imagenum2
0                AJ_Lamas          1                   Zach_Safrin          1
1             Aaron_Guiel          1             Reese_Witherspoon          3
2            Aaron_Tippin          1  Jose_Luis_Rodriguez_Zapatero          1
3  Abdul_Majeed_Shobokshi          1                  Charles_Cope          1
4            Abdullah_Gul         16                     Steve_Cox          1


### Split Into Validation Set

In [5]:
match_dev_train_df, match_dev_val_df = train_test_split(
    match_dev_train_df, test_size=0.2, random_state=42
)
mismatch_dev_train_df, mismatch_dev_val_df = train_test_split(
    mismatch_dev_train_df, test_size=0.2, random_state=42
)

### Loading The Images

In [6]:
# Create a dictionary to store the images
images_memo = {}

In [7]:
transform = transforms.Compose([
    transforms.Resize((250, 250)),       # Resize to match model input
    transforms.ToTensor(),               # Converts to [C, H, W] tensor and scales to [0, 1]
    transforms.Normalize(                # Normalize with ImageNet mean/std if using pretrained model
        mean=[0.485, 0.456, 0.406],      
        std=[0.229, 0.224, 0.225]
    )
])

def load_mismatch_images(person1, image1, person2, image2):
    if(person1, image1) not in images_memo:
        image1_path = os.path.join(images_path, person1, f"{person1}_{image1:04d}.jpg")
        actual_image1 = Image.open(image1_path).convert('RGB')
        transformed = transform(actual_image1)
        images_memo[(person1, image1)] = transformed
    if(person2, image2) not in images_memo:
        image2_path = os.path.join(images_path, person2, f"{person2}_{image2:04d}.jpg")
        actual_image2 = Image.open(image2_path).convert('RGB')
        transformed = transform(actual_image2)
        images_memo[(person2, image2)] = actual_image2
    
    
    return images_memo[(person1, image1)], images_memo[(person2, image2)]


def load_match_images(person, image1, image2):
    return load_mismatch_images(person, image1, person, image2)

In [8]:
def make_minibatches(df, batch_size):
    """
    Create minibatches from the dataframe.
    """
    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i + batch_size]
        images1 = []
        images2 = []
        labels = []
        
        for _, row in batch_df.iterrows():
            if len(row) == 3:
                image1, image2 = load_match_images(row['name'], row['imagenum1'], row['imagenum2'])
                label = 1
            elif len(row) == 4:
                image1, image2 = load_mismatch_images(row['name1'], row['imagenum1'], row['name2'], row['imagenum2'])
                label = 0
            else:
                print(f"Unknown row format: {row}")
                continue
            # Assuming load functions return images as tensors
            images1.append(image1)
            images2.append(image2)
            labels.append(label)
        
        # Convert lists to tensors and yield them as a tuple
        yield torch.stack(images1), torch.stack(images2), torch.tensor(labels)


## Initializing The Network

In [13]:
class SiameseCNN(nn.Module):
    def __init__(self):
        super(SiameseCNN, self).__init__()
        
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=10),     #(1, 250, 250) → (64, 241, 241)
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),                      #(64, 120, 120)

            nn.Conv2d(64, 128, kernel_size=7),    #(128, 114, 114)
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),                      #(128, 57, 57)

            nn.Conv2d(128, 128, kernel_size=4),   #(128,54, 54)
            nn.ReLU(),
            nn.MaxPool2d(2),                      #(128, 27, 27)

            nn.Conv2d(128, 256, kernel_size=4),   #(256, 24, 24)
            nn.ReLU(),
            nn.Flatten()
        )

    def forward(self, X):
        return self.cnn(X)
    
    


class TwinsCNN(nn.Module):
    def __init__(self):
        super(TwinsCNN, self).__init__()
        
        # Share the same SiameseCNN instance for both branches
        self.scnn1 = SiameseCNN()
        self.scnn2 = self.scnn1  # Use the same SiameseCNN instance for shared weights

        self.fc = nn.Sequential(
            nn.Linear(256 * 24 * 24, 1),
            nn.Sigmoid()  # As used in the original paper
        )
        
    def forward(self, X1, X2):
        out1 = self.scnn1(X1)
        out2 = self.scnn2(X2)  # Same weights are used here due to reference
        out = torch.abs(out1 - out2)
        out = out.view(out.size(0), -1)
        return self.fc(out)
    
    def predict(self, X1, X2):
        out1 = self.scnn1(X1)
        out2 = self.scnn2(X2)  # Same weights are used here due to reference
        out = torch.abs(out1 - out2)
        out = out.view(out.size(0), -1)
        return torch.sigmoid(out)


## Training Method

In [16]:
def l2_regularization(model, lambda_l2=0.01):
    l2_loss = 0.0
    for param in model.parameters():
        l2_loss += torch.sum(param ** 2)  # L2 regularization (sum of squared parameters)
    return lambda_l2 * l2_loss


def train(model, train_loader, val_loader, loss_fn, optimizer, device, l2_lambda=0.01, epochs=10):
    model.to(device)
    train_losses = []
    val_losses = []
    val_accuracies = []
    
    for epoch in range(epochs):
        model.train()  # set to training mode
        train_loss = 0
        num_batches = 0
        for inputs1, inputs2, targets in train_loader:  # inputs1 and inputs2 are the image pairs
            inputs1, inputs2, targets = inputs1.to(device), inputs2.to(device), targets.to(device)
            num_batches += 1
            optimizer.zero_grad()               
            outputs = model(inputs1, inputs2)  # Pass both inputs (image pairs) to the model
            loss = loss_fn(outputs, targets) + l2_regularization(model, l2_lambda) 
            loss.backward()                     
            optimizer.step()                    

            train_loss += loss.item()

        avg_train_loss = train_loss / num_batches

        # Validation
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            num_batches = 0
            for inputs1, inputs2, targets in val_loader:
                inputs1, inputs2, targets = inputs1.to(device), inputs2.to(device), targets.to(device)
                num_batches += 1
                outputs = model(inputs1, inputs2)
                loss = loss_fn(outputs, targets) + l2_regularization(model, l2_lambda)
                val_loss += loss.item()

                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

        avg_val_loss = val_loss / num_batches
        accuracy = 100. * correct / total
        val_accuracies.append(accuracy)
        val_losses.append(avg_val_loss)
        train_losses.append(avg_train_loss)

        print(f"Epoch {epoch+1}/{epochs} "
              f"Train Loss: {avg_train_loss:.4f} "
              f"Val Loss: {avg_val_loss:.4f} "
              f"Val Accuracy: {accuracy:.2f}%")

    return train_losses, val_losses, val_accuracies


## Actual Training

In [18]:
model = TwinsCNN()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 128
train_loader = make_minibatches(match_dev_train_df, batch_size)
val_loader = make_minibatches(match_dev_val_df, batch_size)


optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.BCELoss()
train_losses, val_losses, val_accuracies = train(
    model,
    train_loader,
    val_loader,
    loss_fn,
    optimizer,
    device,
    l2_lambda=0.01,
    epochs=10
)


TypeError: expected Tensor as element 69 in argument 0, but got Image